GPT's

works but hyphens etc fuck with the words

In [11]:
import fitz  # PyMuPDF
from docx import Document
from docx.shared import RGBColor
import spacy
from spacy import displacy
from langdetect import detect
from googletrans import Translator

# Load SpaCy models
nlp_en = spacy.load("en_core_web_sm")
nlp_ur = spacy.load("xx_ent_wiki_sm")  # Multilingual model, as there's no dedicated Urdu model
translator = Translator()

def detect_language(text):
    return detect(text)

def translate_to_english(text):
    return translator.translate(text, src='ur', dest='en').text

def translate_to_urdu(text):
    return translator.translate(text, src='en', dest='ur').text

def apply_ner(text, language):
    if language == 'en':
        doc = nlp_en(text)
    else:
        doc = nlp_ur(text)
    entities = [{'text': ent.text, 'label': ent.label_} for ent in doc.ents]
    return entities

def get_entities(text):
    language = detect_language(text)
    original_text = text

    if language == 'ur':
        text = translate_to_english(text)

    entities = apply_ner(text, 'en')

    if language == 'ur':
        translated_entities = []
        for entity in entities:
            translated_text = translate_to_urdu(entity['text'])
            translated_entities.append({'text': translated_text, 'label': entity['label']})
        entities = translated_entities

    return entities

def get_color(label):
    colors = {
        'PERSON': RGBColor(255, 0, 0),
        'NORP': RGBColor(0, 255, 0),
        'FAC': RGBColor(0, 0, 255),
        'ORG': RGBColor(255, 255, 0),
        'GPE': RGBColor(255, 0, 255),
        'LOC': RGBColor(0, 255, 255),
        'PRODUCT': RGBColor(255, 192, 203),
        'EVENT': RGBColor(139, 69, 19),
        'WORK_OF_ART': RGBColor(169, 169, 169),
        'LAW': RGBColor(0, 0, 0),
        'LANGUAGE': RGBColor(255, 255, 255),
        'DATE': RGBColor(0, 255, 255),
        'TIME': RGBColor(255, 0, 255),
        'PERCENT': RGBColor(211, 211, 211),
        'MONEY': RGBColor(105, 105, 105),
        'QUANTITY': RGBColor(173, 216, 230),
        'ORDINAL': RGBColor(144, 238, 144),
        'CARDINAL': RGBColor(255, 255, 224)
    }
    return colors.get(label, RGBColor(0, 0, 0))

def extract_ne(pdf_path, output_path):
    pdf_document = fitz.open(pdf_path)
    doc = Document()

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text = page.get_text()

        entities_list = get_entities(text)
        current_word = ''
        
        paragraph = doc.add_paragraph()
        
        for char in text:
            if char.isalpha():
                current_word += char
            else:
                if current_word:
                    matched_entity = next((entity for entity in entities_list if entity['text'] == current_word), None)
                    if matched_entity:
                        run = paragraph.add_run(current_word)
                        run.font.color.rgb = get_color(matched_entity['label'])
                    else:
                        paragraph.add_run(current_word)
                    current_word = ''
                if char.isspace():
                    paragraph.add_run(char)
                else:
                    run = paragraph.add_run(char)
                    run.font.color.rgb = RGBColor(0, 0, 0)

    doc.save(output_path)
    print("Named entities extracted and saved to", output_path)


In [ ]:
import fitz  # PyMuPDF
from docx import Document
from docx.shared import RGBColor
import spacy
from spacy import displacy
from langdetect import detect
from google.cloud import translate_v2 as translate

# Load SpaCy models
nlp_en = spacy.load("en_core_web_sm")
nlp_ur = spacy.load("xx_ent_wiki_sm")  # Multilingual model, as there's no dedicated Urdu model
translator = translate.Client()

def detect_language(text):
    return detect(text)

def translate_to_english(text):
    result = translator.translate(text, source_language='ur', target_language='en')
    return result['translatedText']

def translate_to_urdu(text):
    result = translator.translate(text, source_language='en', target_language='ur')
    return result['translatedText']

def apply_ner(text, language):
    if language == 'en':
        doc = nlp_en(text)
    else:
        doc = nlp_ur(text)
    entities = [{'text': ent.text, 'label': ent.label_} for ent in doc.ents]
    return entities

def get_entities(text):
    language = detect_language(text)
    original_text = text

    if language == 'ur':
        text = translate_to_english(text)

    entities = apply_ner(text, 'en')

    if language == 'ur':
        translated_entities = []
        for entity in entities:
            translated_text = translate_to_urdu(entity['text'])
            translated_entities.append({'text': translated_text, 'label': entity['label']})
        entities = translated_entities

    return entities

def get_color(label):
    colors = {
        'PERSON': RGBColor(255, 0, 0),
        'NORP': RGBColor(0, 255, 0),
        'FAC': RGBColor(0, 0, 255),
        'ORG': RGBColor(255, 255, 0),
        'GPE': RGBColor(255, 0, 255),
        'LOC': RGBColor(0, 255, 255),
        'PRODUCT': RGBColor(255, 192, 203),
        'EVENT': RGBColor(139, 69, 19),
        'WORK_OF_ART': RGBColor(169, 169, 169),
        'LAW': RGBColor(0, 0, 0),
        'LANGUAGE': RGBColor(255, 255, 255),
        'DATE': RGBColor(0, 255, 255),
        'TIME': RGBColor(255, 0, 255),
        'PERCENT': RGBColor(211, 211, 211),
        'MONEY': RGBColor(105, 105, 105),
        'QUANTITY': RGBColor(173, 216, 230),
        'ORDINAL': RGBColor(144, 238, 144),
        'CARDINAL': RGBColor(255, 255, 224)
    }
    return colors.get(label, RGBColor(0, 0, 0))

def extract_ne(pdf_path, output_path):
    pdf_document = fitz.open(pdf_path)
    doc = Document()

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text = page.get_text()

        entities_list = get_entities(text)
        current_word = ''
        
        paragraph = doc.add_paragraph()
        
        for char in text:
            if char.isalpha():
                current_word += char
            else:
                if current_word:
                    matched_entity = next((entity for entity in entities_list if entity['text'] == current_word), None)
                    if matched_entity:
                        run = paragraph.add_run(current_word)
                        run.font.color.rgb = get_color(matched_entity['label'])
                    else:
                        paragraph.add_run(current_word)
                    current_word = ''
                if char.isspace():
                    paragraph.add_run(char)
                else:
                    run = paragraph.add_run(char)
                    run.font.color.rgb = RGBColor(0, 0, 0)

    doc.save(output_path)
    print("Named entities extracted and saved to", output_path)


In [ ]:
extract_ne('urdu.pdf','urduOutput.docx')

In [12]:
from flask import Flask, render_template, request, send_file
import os
from NER import extract_ne  # Ensure this import is correct

app = Flask(__name__)
UPLOAD_FOLDER = 'uploads'
OUTPUT_FOLDER = 'outputs'

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['OUTPUT_FOLDER'] = OUTPUT_FOLDER

# Ensure the directories exist
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/extract_ne', methods=['POST'])
def extract_ne_route():
    if 'file' not in request.files:
        return 'No file part'
    file = request.files['file']
    if file.filename == '':
        return 'No selected file'
    if file:
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)
        
        output_path = os.path.join(app.config['OUTPUT_FOLDER'], 'colored_text.docx')
        extract_ne(file_path, output_path)
        
        return send_file(output_path, as_attachment=True)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1